In [1]:
!git clone https://github.com/lucidrains/video-diffusion-pytorch.git

Cloning into 'video-diffusion-pytorch'...
remote: Enumerating objects: 402, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 402 (delta 187), reused 124 (delta 124), pack-reused 211
Receiving objects: 100% (402/402), 1.30 MiB | 30.88 MiB/s, done.
Resolving deltas: 100% (298/298), done.


In [2]:
! pip install video-diffusion-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 5.5 MB/s eta 0:00:00


In [3]:
import torch
from video_diffusion_pytorch import Unet3D, GaussianDiffusion

model = Unet3D(
    dim = 64,
    dim_mults = (1, 2, 4, 8)
)

diffusion = GaussianDiffusion(
    model,
    image_size = 32,
    num_frames = 5,
    timesteps = 1000,   # number of steps
    loss_type = 'l1'    # L1 or L2
)

videos = torch.randn(1, 3, 5, 32, 32) # video (batch, channels, frames, height, width) - normalized from -1 to +1
loss = diffusion(videos)
loss.backward()
# after a lot of training

sampled_videos = diffusion.sample(batch_size = 4)
sampled_videos.shape # (4, 3, 5, 32, 32)

sampling loop time step: 100%|██████████| 1000/1000 [40:28<00:00,  2.43s/it]


torch.Size([4, 3, 5, 32, 32])

In [4]:
import torch
from video_diffusion_pytorch import Unet3D, GaussianDiffusion

model = Unet3D(
    dim = 64,
    cond_dim = 64,
    dim_mults = (1, 2, 4, 8)
)

diffusion = GaussianDiffusion(
    model,
    image_size = 32,
    num_frames = 5,
    timesteps = 1000,   # number of steps
    loss_type = 'l1'    # L1 or L2
)

videos = torch.randn(2, 3, 5, 32, 32) # video (batch, channels, frames, height, width)
text = torch.randn(2, 64)             # assume output of BERT-large has dimension of 64

loss = diffusion(videos, cond = text)
loss.backward()
# after a lot of training

sampled_videos = diffusion.sample(cond = text)
sampled_videos.shape # (2, 3, 5, 32, 32)

sampling loop time step: 100%|██████████| 1000/1000 [15:19<00:00,  1.09it/s]


torch.Size([2, 3, 5, 32, 32])

In [5]:
!git clone https://github.com/cvdfoundation/kinetics-dataset.git


Cloning into 'kinetics-dataset'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 146 (delta 41), reused 30 (delta 25), pack-reused 84
Receiving objects: 100% (146/146), 38.91 KiB | 12.97 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [6]:
!cd kinetics-dataset

In [7]:
!bash /content/kinetics-dataset/k600_downloader.sh

--2023-03-24 17:39:32--  https://s3.amazonaws.com/kinetics/600/train/k600_train_path.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.26.230, 52.217.168.0, 54.231.202.88, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.26.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39809 (39K) [text/plain]
Saving to: ‘k600_targz/train/k600_train_path.txt’

k600_train_path.txt 100%[===================>]  38.88K   174KB/s    in 0.2s    

2023-03-24 17:39:33 (174 KB/s) - ‘k600_targz/train/k600_train_path.txt’ saved [39809/39809]

--2023-03-24 17:39:33--  https://s3.amazonaws.com/kinetics/600/train/abseiling.tar.gz
Reusing existing connection to s3.amazonaws.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 2000227200 (1.9G) [application/x-tar]
Saving to: ‘k600_targz/train/abseiling.tar.gz’

abseiling.tar.gz    100%[===================>]   1.86G  13.1MB/s    in 2m 43s  

2023-03-24 17:42:16 (11.7 MB/s) - ‘k600_targz/train/abseiling.tar

In [8]:
!bash /content/kinetics-dataset/k600_extractor.sh

mv: 'k600_targz/train/k600_train_path.txt' and 'k600_targz/train/k600_train_path.txt' are the same file
mv: 'k600_targz/train/applauding.tar.gz' and 'k600_targz/train/applauding.tar.gz' are the same file
mv: 'k600_targz/train/abseiling.tar.gz' and 'k600_targz/train/abseiling.tar.gz' are the same file
Extracting k600_targz/train/abseiling.tar.gz to k600/train
Extracting k600_targz/train/acting_in_play.tar.gz to k600/train
Extracting k600_targz/train/adjusting_glasses.tar.gz to k600/train
Extracting k600_targz/train/air_drumming.tar.gz to k600/train
Extracting k600_targz/train/alligator_wrestling.tar.gz to k600/train
Extracting k600_targz/train/answering_questions.tar.gz to k600/train
Extracting k600_targz/train/applauding.tar.gz to k600/train

gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now
find: ‘k600_targz/val’: No such file or directory
ls: cannot access 'k600_targz/val': No such file or dire

**Training**

In [ ]:
import torch
from video_diffusion_pytorch import Unet3D, GaussianDiffusion, Trainer

model = Unet3D(
    dim = 64,
    dim_mults = (1, 2, 4, 8),
)

diffusion = GaussianDiffusion(
    model,
    image_size = 64,
    num_frames = 10,
    timesteps = 1000,   # number of steps
    loss_type = 'l1'    # L1 or L2
).cuda()

trainer = Trainer(
    diffusion,
    '/content/video-diffusion-pytorch/samples',                         # this folder path needs to contain all your training data, as .gif files, of correct image size and number of frames
    train_batch_size = 32,
    train_lr = 1e-4,
    save_and_sample_every = 1000,
    train_num_steps = 700000,         # total training steps
    gradient_accumulate_every = 2,    # gradient accumulation steps
    ema_decay = 0.995,                # exponential moving average decay
    amp = True                        # turn on mixed precision
)

trainer.train()

found 2 videos as gif files at /content/video-diffusion-pytorch/samples
0: 0.8529263138771057
0: 0.8497949242591858
1: 0.8223056197166443
1: 0.8184053301811218
2: 0.8060834407806396
2: 0.813481330871582
3: 0.778141438961029
3: 0.8083300590515137
4: 0.7510265707969666
4: 0.8008109331130981
5: 0.7393338084220886
5: 0.7535209059715271
6: 0.7652549147605896
6: 0.7674854397773743
7: 0.7539202570915222
7: 0.6967055201530457
8: 0.717527449131012
8: 0.7065269947052002
9: 0.6645025610923767
9: 0.6578016877174377
10: 0.7046740055084229
10: 0.6601694822311401
11: 0.679360568523407
11: 0.7146182656288147
12: 0.647083580493927
12: 0.599655032157898
13: 0.599264919757843
13: 0.6743917465209961
14: 0.5940456986427307
14: 0.7079415321350098
15: 0.6697700619697571
15: 0.6663060784339905
16: 0.5528009533882141
16: 0.6197574734687805
17: 0.5461177229881287
17: 0.5242044925689697
18: 0.5721036195755005
18: 0.7128007411956787
19: 0.6085557341575623
19: 0.5623230338096619
20: 0.46244698762893677
20: 0.61689

In [ ]:
loss = diffusion(videos, cond = text, prob_focus_present = 0.5) # for 50% of videos, focus on the present during training
loss.backward()